In [1]:
targetFolderHBK = r"G:\Semester Arbeit\Programming\Extracted_Features\HBK\HBK_14285Hz_original_all_features\features"
targetFolderMCC5 = r"G:\Semester Arbeit\Programming\Extracted_Features\MCC5\MCC5_12800Hz_original_all_features_motor_vibration_x\features"
targetFolderSIZA = r"G:\Semester Arbeit\Programming\Extracted_Features\SIZA\SIZA_original_all_features\features"
normalization_method = "z_score"

# Environment Setup & Imports

In [2]:
import os
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import numpy as np
import mlflow
from pytorch_lightning.loggers import MLFlowLogger
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, hamming_loss, hinge_loss, jaccard_score, log_loss, precision_score, recall_score, f1_score, make_scorer
from pathlib import Path
from pycaret.classification import * 
from torch import tensor
from torchmetrics.classification import BinaryAccuracy, MulticlassAccuracy
import optuna
import torch
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig, CategoryEmbeddingModel,GatedAdditiveTreeEnsembleConfig, NodeConfig, FTTransformerConfig, TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    ModelConfig,
    TrainerConfig,
    ExperimentConfig,
)
from collections import Counter
from data_loader import load_feature_data

## Helper Functions

In [3]:
def normalizeDataframe(dataframe, normalization_method):
    """
    Normalizes the features of a dataframe using a specified method.

    Args:
        dataframe (pd.DataFrame): The input dataframe with a 'Label' column.
        normalization_method (str): The method to use ("min_max", "z_score", "robust_scaling").

    Returns:
        pd.DataFrame: The dataframe with scaled features.
    """
    # Separate features (X) and the target variable (y)
    y = dataframe['Label']
    X = dataframe.drop(columns=['Label'])

    # Select the scaler based on the chosen method
    if normalization_method == "min_max":
        scaler = MinMaxScaler()
    elif normalization_method == "z_score":
        scaler = StandardScaler()
    elif normalization_method == "robust_scaling":
        scaler = RobustScaler()
    else:
        # Raise an error for an invalid method name
        raise ValueError(f"Unknown normalization_method: '{normalization_method}'")

    # Fit the scaler to the data and transform it
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns,
        index=X.index
    )

    # Rejoin the scaled features with the label column
    df_scaled = X_scaled.join(y)
    
    return df_scaled

In [4]:
def plotPredictionHistograms(df, domain, normalization):
    # 1) mark correct vs incorrect
    df = df.copy()
    df['prediction_quality'] = np.where(
        df['Label'] == df['prediction_label'],
        'correct',
        'incorrect'
    )
    
    # 2) choose a palette (you can override these colors if you like)
    pal = dict(zip(
        ['correct','incorrect'],
        sns.color_palette(n_colors=2)
    ))
    
    skip = {'Label','prediction_label','prediction_score','prediction_quality'}
    for col in df.columns:
        if col in skip:
            continue
        
        fig, ax = plt.subplots(figsize=(8,4))
        sns.histplot(
            data=df, x=col, hue='prediction_quality',
            palette=pal,
            kde=True, multiple='layer', element='step',
            alpha=0.5,
            ax=ax
        )
        
        # 3) build a manual legend using the same palette
        handles = [
            mpatches.Patch(color=pal[k], label=k)
            for k in ['correct','incorrect']
        ]
        ax.legend(
            handles=handles,
            title='Prediction Quality'
        )
        
        ax.set_title(
            f"Distribution of {col} in the '{domain}' domain\n"
            f"(normalization = '{normalization}')"
        )
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        plt.tight_layout()
        plt.show()

In [5]:
def get_incorrect_predictions(df):
    return df[
        ((df['Label'] == 'damaged')   & (df['prediction_label'] == 'healthy'))
      | ((df['Label'] == 'healthy')  & (df['prediction_label'] == 'damaged'))
    ].copy()

In [6]:
def get_feature_importance_df(model, df):
    importance = model.feature_importances_
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': importance
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

In [7]:
def get_svm_feature_importance_df(model, df):
    if not hasattr(model, 'coef_'):
        raise ValueError("This SVM model has no coefficients. Use a linear kernel.")
    
    importance = model.coef_.ravel()  # Flatten in case of binary classification
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': abs(importance)
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)


In [8]:
feature_counter = Counter()
def add_top_features(feature_df: pd.DataFrame, top_n: int):
    top_features = feature_df.nlargest(top_n, 'importance')['Features']
    feature_counter.update(top_features)
    
def plot_feature_importance():
    feature_freq = pd.DataFrame(feature_counter.items(), columns=['Feature', 'Count'])
    plt.figure(figsize=(10, 5))
    sns.barplot(data=feature_freq.sort_values(by='Count', ascending=False),
                x='Feature', y='Count')
    plt.xticks(rotation=45)
    plt.title('Feature Frequency Across Experiments')
    plt.tight_layout()
    plt.show()

## Import Dataset

In [9]:
experiment_name = "Best_Hyperparameters_z_score"

In [10]:
df_binary_HBK = load_feature_data(
    features_path=targetFolderHBK,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 161 files into a DataFrame with shape (200093, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (200093, 28)


In [11]:
df_binary_SIZA = load_feature_data(
    features_path=targetFolderSIZA,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [12]:
df_binary_MCC5 = load_feature_data(
    features_path=targetFolderMCC5,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [13]:
combined_df = pd.concat([df_binary_HBK, df_binary_SIZA, df_binary_MCC5], ignore_index=True)

In [14]:
normalized_df = normalizeDataframe(combined_df, normalization_method)

In [15]:
features_df_training_normalized, features_df_testing_normalized = train_test_split(
    normalized_df, 
    test_size=0.2,    # e.g., 20% for testing
    random_state=42   # for reproducibility
)

# Experiment Setup (ML)

## Setup Hyperparameters

In [16]:
experiment = setup(features_df_training_normalized, target='Label', log_experiment = True, experiment_name = experiment_name, use_gpu = True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Description,Value
0,Session id,1768
1,Target,Label
2,Target type,Binary
3,Target mapping,"damaged: 0, healthy: 1"
4,Original data shape,"(246359, 28)"
5,Transformed data shape,"(246359, 28)"
6,Transformed train set shape,"(172451, 28)"
7,Transformed test set shape,"(73908, 28)"
8,Numeric features,27
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

## Add aditional metrics

In [17]:
# Binary classification metrics
add_metric('balanced_acc', 'Balance Acc', balanced_accuracy_score, target='pred', greater_is_better=True)
add_metric('hamming_loss', 'Hamming Loss', hamming_loss, target='pred', greater_is_better=False)
add_metric('jaccard_score', 'Jaccard Score', jaccard_score, target='pred', greater_is_better=True)
add_metric('log_loss', 'Log Loss', log_loss, target='pred_proba', greater_is_better=False)

Name                                                          Log Loss
Display Name                                                  Log Loss
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: log_loss, dtype: object

In [18]:
all_metrics = get_metrics()
all_metrics

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000002716C87BA30>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000002716C87B...,"make_scorer(cohen_kappa_score, response_method...",pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000002716C8E0...,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False
balanced_acc,Balance Acc,Balance Acc,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(balanced_accuracy_score, response_...",pred,{},True,True,True
hamming_loss,Hamming Loss,Hamming Loss,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(hamming_loss, greater_is_better=Fa...",pred,{},False,True,True


In [19]:
num_iterations_tuning = 10
optimized_metric = 'F1'

## Light Gradient Boosting Machine

In [20]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9498,0.9859,0.9498,0.9511,0.9480,0.8464,0.8531,0.8959,0.0502,0.7817,0.1186
1,0.9546,0.9867,0.9546,0.9554,0.9532,0.8622,0.8672,0.9072,0.0454,0.8028,0.1119
2,0.9511,0.9866,0.9511,0.9523,0.9493,0.8503,0.8568,0.8983,0.0489,0.7868,0.1161
3,0.9542,0.9879,0.9542,0.9552,0.9528,0.8609,0.8662,0.9058,0.0458,0.8009,0.1103
4,0.9538,0.9867,0.9538,0.9548,0.9523,0.8596,0.8650,0.9049,0.0462,0.7991,0.1126
5,0.9508,0.9859,0.9508,0.9519,0.9490,0.8495,0.8558,0.8984,0.0492,0.7860,0.1170
6,0.9517,0.9865,0.9517,0.9526,0.9501,0.8528,0.8585,0.9010,0.0483,0.7903,0.1150
7,0.9539,0.9874,0.9539,0.9548,0.9524,0.8598,0.8651,0.9053,0.0461,0.7995,0.1116
8,0.9519,0.9856,0.9519,0.9527,0.9503,0.8534,0.8590,0.9017,0.0481,0.7912,0.1173


2025/09/26 11:36:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [21]:
lightgbm_tuned_model, lightgbm_tuner = tune_model(lightgbm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9737,0.9929,0.9737,0.9736,0.9734,0.9231,0.9237,0.9531,0.0263,0.8868,0.0804
1,0.9775,0.9936,0.9775,0.9774,0.9773,0.9345,0.9349,0.9598,0.0225,0.9028,0.0730
2,0.9739,0.9933,0.9739,0.9738,0.9737,0.9238,0.9244,0.9535,0.0261,0.8877,0.0791
3,0.9761,0.9938,0.9761,0.9760,0.9759,0.9305,0.9308,0.9583,0.0239,0.8972,0.0738
4,0.9756,0.9936,0.9756,0.9755,0.9755,0.9291,0.9295,0.9577,0.0244,0.8953,0.0746
5,0.9734,0.9931,0.9734,0.9733,0.9732,0.9224,0.9230,0.9522,0.0266,0.8856,0.0810
6,0.9744,0.9927,0.9744,0.9742,0.9742,0.9253,0.9257,0.9553,0.0256,0.8899,0.0831
7,0.9766,0.9934,0.9766,0.9766,0.9764,0.9318,0.9323,0.9577,0.0234,0.8989,0.0765
8,0.9731,0.9926,0.9731,0.9729,0.9729,0.9216,0.9220,0.9536,0.0269,0.8848,0.0832


[LightGBM] [Warning] feature_fraction is set=0.9154130135317432, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9154130135317432
[LightGBM] [Warning] bagging_fraction is set=0.8015013984914804, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8015013984914804
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9154130135317432, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9154130135317432
[LightGBM] [Warning] bagging_fraction is set=0.8015013984914804, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8015013984914804
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Info] Number of positive: 34940, number of negative: 120265
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6885
[LightGBM] [Info] Number of data

2025/09/26 11:53:44 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [28]:
print(lightgbm_tuned_model)

LGBMClassifier(bagging_fraction=0.8455104741417432, bagging_freq=0,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', feature_fraction=0.42947671039481,
               importance_type='split', learning_rate=0.47910287588557177,
               max_depth=-1, min_child_samples=41, min_child_weight=0.001,
               min_split_gain=0.23016472358397666, n_estimators=245, n_jobs=-1,
               num_leaves=250, objective=None, random_state=1768,
               reg_alpha=0.00039695268896179983,
               reg_lambda=3.6071600095045767e-06, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [60]:
predictions_lightgbm = predict_model(lightgbm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9781,0.9943,0.9781,0.9780,0.9780,0.9366,0.9369,0.9617,0.0219,0.9060,0.0704


[LightGBM] [Warning] feature_fraction is set=0.42947671039481, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42947671039481
[LightGBM] [Warning] bagging_fraction is set=0.8455104741417432, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8455104741417432
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.42947671039481, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42947671039481
[LightGBM] [Warning] bagging_fraction is set=0.8455104741417432, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8455104741417432
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


In [22]:
evaluate_model(lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [23]:
lightgbm_top_features = get_feature_importance_df(lightgbm, features_df_training_normalized)
lightgbm_top_features

,Features,importance
0,bandwidth,320
1,meanFreq,303
2,meanWavelet,301
3,medianFreq,295
4,spectral_flatness,266
5,skewness,184
6,entropyWavelet,167
7,spectral_skewness,163
8,mean,107
9,spectral_kurtosis,105


In [24]:
predictions_lightgbm = predict_model(lightgbm, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9522,0.9866,0.9522,0.9532,0.9506,0.8549,0.8606,0.9023,0.0478,0.7933,0.1148


In [25]:
predictions_lightgbm

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
93960,0.235183,1.062654,1.181320,0.246260,-0.055801,1.856693,0.318572,0.229234,1.887167,0.169256,...,0.175728,0.144233,0.515128,0.251476,-0.118528,-0.143934,-0.117957,damaged,damaged,0.9999
371,0.012873,-0.573892,-0.608301,0.586265,-0.043286,-0.772690,0.000642,-0.026207,-0.986468,0.022755,...,-0.276503,0.164883,-0.302537,-0.241933,-0.119511,-0.156183,-0.119637,healthy,damaged,0.8974
33266,0.068736,-0.612422,-0.648066,-0.205265,-0.089285,-0.831232,-0.524111,-0.441818,-1.303345,-0.235899,...,-0.276743,0.163780,-0.303045,-0.242338,-0.119511,-0.156183,-0.119637,healthy,healthy,0.8038
238774,0.033096,-0.136916,-0.127309,0.167115,-0.073693,-0.051146,0.591589,0.398757,0.412783,-0.002132,...,-0.210708,0.202637,-0.225960,-0.198761,-0.119454,-0.155680,-0.119575,damaged,damaged,0.9226
210317,0.033394,-0.435811,-0.453998,0.078050,-0.009010,-0.571356,-0.224380,-0.089825,-0.517015,0.712591,...,-0.266166,0.174123,-0.291462,-0.236005,-0.119508,-0.156153,-0.119633,damaged,damaged,0.8891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241132,0.036482,-0.427159,-0.444468,-0.047157,0.270062,-0.459822,0.946953,0.994946,-0.094569,1.758543,...,-0.264591,0.173715,-0.290368,-0.234696,-0.119505,-0.156126,-0.119630,damaged,damaged,0.8011
214791,0.035452,-0.406114,-0.421478,-0.047884,0.268881,-0.413760,0.859693,0.815125,-0.085377,1.173851,...,-0.262673,0.175848,-0.287531,-0.232590,-0.119501,-0.156096,-0.119626,damaged,damaged,0.8678
165165,0.112750,0.319713,0.371290,0.497024,0.115964,0.739326,0.687841,0.680583,1.376881,1.164648,...,0.002140,0.136958,-0.019808,-0.080584,-0.116263,-0.139601,-0.116393,damaged,damaged,0.9998
217690,0.038995,0.164117,0.201626,0.444060,-0.093651,0.278279,-0.162111,-0.169209,0.501702,-0.145207,...,-0.105311,0.204984,-0.101980,-0.121442,-0.118854,-0.151695,-0.118953,damaged,damaged,0.7371


In [26]:
get_incorrect_predictions(predictions_lightgbm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
371,0.012873,-0.573892,-0.608301,0.586265,-0.043286,-0.772690,0.000642,-0.026207,-0.986468,0.022755,...,-0.276503,0.164883,-0.302537,-0.241933,-0.119511,-0.156183,-0.119637,healthy,damaged,0.8974
258524,0.036464,-0.602333,-0.637307,-0.407115,0.131417,-0.806982,0.949561,0.854643,-1.057003,1.018010,...,-0.276646,0.164046,-0.302923,-0.242255,-0.119511,-0.156183,-0.119637,healthy,damaged,0.6787
207745,0.036146,-0.054912,-0.037686,-0.206209,-0.191703,-0.166550,-0.817718,-0.706493,-0.152971,-0.686273,...,-0.178280,0.195415,-0.197872,-0.181161,-0.119248,-0.154350,-0.119369,healthy,damaged,0.8491
255723,0.035643,-0.187058,-0.182074,0.147473,-0.086570,-0.243538,-0.400318,-0.316864,-0.104728,0.027741,...,-0.222601,0.196681,-0.240971,-0.205668,-0.119463,-0.155770,-0.119586,healthy,damaged,0.6397
207040,0.039656,-0.082632,-0.067949,0.162263,-0.036778,0.060171,0.134858,0.100614,0.324422,0.206646,...,-0.195858,0.206877,-0.207824,-0.184564,-0.119401,-0.155277,-0.119519,healthy,damaged,0.6681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206916,0.041697,-0.158916,-0.151280,0.231444,-0.101904,-0.181929,-0.347732,-0.291088,-0.043652,-0.061274,...,-0.216526,0.200570,-0.232714,-0.198674,-0.119441,-0.155599,-0.119563,healthy,damaged,0.9436
200497,0.033159,-0.386435,-0.400006,0.047990,0.064846,-0.452461,0.219733,0.307008,-0.226820,1.091975,...,-0.258714,0.176503,-0.284612,-0.231184,-0.119501,-0.156092,-0.119626,healthy,damaged,0.5748
258687,0.037649,-0.412417,-0.428330,0.099707,0.062800,-0.512874,-0.121624,-0.001592,-0.423838,0.784442,...,-0.262370,0.174837,-0.288400,-0.233108,-0.119503,-0.156112,-0.119628,healthy,damaged,0.6131
205706,0.035491,-0.426794,-0.444088,0.048360,0.038563,-0.511299,0.161131,0.161734,-0.395557,0.467480,...,-0.264963,0.174827,-0.290323,-0.234408,-0.119506,-0.156138,-0.119632,healthy,damaged,0.6595


## Random Forest Classifier

In [29]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9746,0.9951,0.9746,0.9749,0.9742,0.9249,0.9265,0.9482,0.0254,0.8886,0.0769
1,0.9788,0.9958,0.9788,0.9790,0.9785,0.9376,0.9386,0.9569,0.0212,0.9068,0.0718
2,0.9741,0.9950,0.9741,0.9744,0.9736,0.9233,0.9249,0.9470,0.0259,0.8863,0.0783
3,0.9772,0.9960,0.9772,0.9773,0.9768,0.9328,0.9339,0.9545,0.0228,0.9000,0.0708
4,0.9780,0.9958,0.9780,0.9782,0.9776,0.9351,0.9363,0.9549,0.0220,0.9032,0.0748
5,0.9764,0.9954,0.9764,0.9767,0.9760,0.9303,0.9317,0.9515,0.0236,0.8963,0.0760
6,0.9746,0.9954,0.9746,0.9747,0.9742,0.9251,0.9264,0.9499,0.0254,0.8891,0.0746
7,0.9779,0.9956,0.9779,0.9781,0.9776,0.9350,0.9361,0.9552,0.0221,0.9030,0.0737
8,0.9756,0.9948,0.9756,0.9759,0.9753,0.9281,0.9295,0.9506,0.0244,0.8932,0.0793


2025/09/26 12:05:01 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [30]:
rf_tuned_model, rf_tuner = tune_model(rf, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9298,0.9630,0.9298,0.9307,0.9264,0.7811,0.7912,0.8590,0.0702,0.7008,0.2332
1,0.9382,0.9656,0.9382,0.9394,0.9356,0.8089,0.8175,0.8746,0.0618,0.7345,0.2258
2,0.9307,0.9611,0.9307,0.9314,0.9275,0.7848,0.7940,0.8622,0.0693,0.7056,0.2300
3,0.9354,0.9660,0.9354,0.9363,0.9326,0.7999,0.8087,0.8700,0.0646,0.7236,0.2250
4,0.9357,0.9656,0.9357,0.9367,0.9329,0.8008,0.8096,0.8704,0.0643,0.7246,0.2243
5,0.9323,0.9647,0.9323,0.9330,0.9292,0.7898,0.7989,0.8648,0.0677,0.7115,0.2264
6,0.9345,0.9663,0.9345,0.9355,0.9315,0.7966,0.8058,0.8679,0.0655,0.7195,0.2249
7,0.9341,0.9635,0.9341,0.9349,0.9312,0.7957,0.8045,0.8680,0.0659,0.7186,0.2274
8,0.9329,0.9644,0.9329,0.9341,0.9297,0.7910,0.8010,0.8640,0.0671,0.7125,0.2277


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


2025/09/26 13:00:12 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [31]:
print(rf_tuned_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=1768, verbose=0,
                       warm_start=False)


In [61]:
predictions_rf = predict_model(rf_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9785,0.9961,0.9785,0.9786,0.9782,0.9369,0.9379,0.9569,0.0215,0.9059,0.0725


In [32]:
evaluate_model(rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [33]:
rf_top_features = get_feature_importance_df(rf, features_df_training_normalized)
rf_top_features

,Features,importance
0,meanWavelet,0.113125
1,medianFreq,0.080381
2,energyWavelet,0.071475
3,meanFreq,0.057107
4,spectral_entropy,0.054538
5,spectral_flatness,0.049845
6,bandwidth,0.046300
7,rms,0.045366
8,std,0.043982
9,meanSpectrogram,0.040018


In [34]:
predictions_rf = predict_model(rf, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9785,0.9961,0.9785,0.9786,0.9782,0.9369,0.9379,0.9569,0.0215,0.9059,0.0725


In [35]:
get_incorrect_predictions(predictions_rf)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
258524,0.036464,-0.602333,-0.637307,-0.407115,0.131417,-0.806982,0.949561,0.854643,-1.057003,1.018010,...,-0.276646,0.164046,-0.302923,-0.242255,-0.119511,-0.156183,-0.119637,healthy,damaged,0.78
207745,0.036146,-0.054912,-0.037686,-0.206209,-0.191703,-0.166550,-0.817718,-0.706493,-0.152971,-0.686273,...,-0.178280,0.195415,-0.197872,-0.181161,-0.119248,-0.154350,-0.119369,healthy,damaged,0.75
255723,0.035643,-0.187058,-0.182074,0.147473,-0.086570,-0.243538,-0.400318,-0.316864,-0.104728,0.027741,...,-0.222601,0.196681,-0.240971,-0.205668,-0.119463,-0.155770,-0.119586,healthy,damaged,0.63
202856,0.030382,-0.143407,-0.134430,0.103309,-0.026063,-0.108780,-0.042167,-0.014115,0.139272,0.306646,...,-0.212232,0.201269,-0.227989,-0.197034,-0.119436,-0.155564,-0.119558,healthy,damaged,0.90
205260,0.032710,-0.431614,-0.449423,0.020514,0.119952,-0.493347,0.822264,0.709045,-0.204833,0.761597,...,-0.265597,0.174436,-0.290946,-0.234815,-0.119507,-0.156146,-0.119633,healthy,damaged,0.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259068,0.034660,-0.417189,-0.433603,0.038210,0.128827,-0.489982,0.490157,0.524692,-0.231692,1.146661,...,-0.263326,0.174694,-0.289062,-0.233749,-0.119504,-0.156123,-0.119630,healthy,damaged,0.63
240598,0.036515,-0.391927,-0.405950,0.015472,0.119186,-0.474286,0.223008,0.354228,-0.224722,1.366387,...,-0.259031,0.175550,-0.285444,-0.231017,-0.119498,-0.156074,-0.119623,damaged,healthy,0.51
262404,0.025626,0.131088,0.165457,0.391404,-0.025749,0.371228,-0.040433,-0.019632,0.566100,0.261995,...,-0.113738,0.196419,-0.118451,-0.138517,-0.118858,-0.151940,-0.118968,healthy,damaged,0.69
206916,0.041697,-0.158916,-0.151280,0.231444,-0.101904,-0.181929,-0.347732,-0.291088,-0.043652,-0.061274,...,-0.216526,0.200570,-0.232714,-0.198674,-0.119441,-0.155599,-0.119563,healthy,damaged,0.90


## SVM

In [36]:
svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8780,0.8718,0.8780,0.8755,0.8682,0.6015,0.6219,0.7639,0.1220,0.5066,-0.0000
1,0.8845,0.8815,0.8845,0.8802,0.8802,0.6459,0.6512,0.8018,0.1155,0.5595,-0.0000
2,0.8820,0.8722,0.8820,0.8781,0.8747,0.6245,0.6375,0.7813,0.1180,0.5329,-0.0000
3,0.8881,0.8821,0.8881,0.8844,0.8850,0.6624,0.6655,0.8145,0.1119,0.5780,-0.0000
4,0.8891,0.8796,0.8891,0.8852,0.8842,0.6563,0.6636,0.8034,0.1109,0.5680,-0.0000
5,0.8724,0.8765,0.8724,0.8679,0.8630,0.5869,0.6039,0.7598,0.1276,0.4947,-0.0000
6,0.8855,0.8832,0.8855,0.8818,0.8827,0.6564,0.6588,0.8135,0.1145,0.5730,-0.0000
7,0.8765,0.8814,0.8765,0.8736,0.8747,0.6358,0.6367,0.8091,0.1235,0.5558,-0.0000
8,0.8684,0.8734,0.8684,0.8647,0.8660,0.6096,0.6108,0.7946,0.1316,0.5304,-0.0000


2025/09/26 13:00:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [37]:
svm_tuned_model, svm_tuner = tune_model(svm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8920,0.8875,0.8920,0.8901,0.8850,0.6549,0.6698,0.7936,0.1080,0.5617,-0.0000
1,0.8956,0.8941,0.8956,0.8933,0.8897,0.6704,0.6822,0.8042,0.1044,0.5791,-0.0000
2,0.8926,0.8868,0.8926,0.8902,0.8861,0.6590,0.6721,0.7974,0.1074,0.5669,-0.0000
3,0.8942,0.8919,0.8942,0.8916,0.8884,0.6667,0.6779,0.8033,0.1058,0.5758,-0.0000
4,0.8999,0.8968,0.8999,0.8982,0.8941,0.6835,0.6957,0.8100,0.1001,0.5924,-0.0000
5,0.8935,0.8944,0.8935,0.8913,0.8871,0.6621,0.6751,0.7990,0.1065,0.5700,-0.0000
6,0.8964,0.8951,0.8964,0.8943,0.8903,0.6719,0.6844,0.8042,0.1036,0.5803,-0.0000
7,0.8963,0.8933,0.8963,0.8942,0.8903,0.6721,0.6843,0.8045,0.1037,0.5806,-0.0000
8,0.8945,0.8883,0.8945,0.8922,0.8883,0.6657,0.6782,0.8012,0.1055,0.5740,-0.0000


2025/09/26 13:01:46 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [38]:
print(svm_tuned_model)

SGDClassifier(alpha=1.983570694577895e-06, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.050879465480455605,
              fit_intercept=True, l1_ratio=0.3264919553269943,
              learning_rate='adaptive', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=1768, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [62]:
predictions_svm = predict_model(svm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8961,0,0.8961,0.8943,0.8898,0.6710,0.6844,0.8028,0.1039,0.5792,0


In [39]:
evaluate_model(svm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [40]:
svm_top_features = get_svm_feature_importance_df(svm, features_df_training_normalized)
svm_top_features

,Features,importance
0,ptp,7.630259
1,clearance,5.968754
2,entropyWavelet,5.569796
3,entropy,3.389358
4,spectral_entropy,3.356334
5,energy,2.531843
6,impulse,2.116005
7,rms,1.965590
8,meanSpectrogram,1.827582
9,meanWavelet,1.564300


In [41]:
predictions_svm = predict_model(svm, data=features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8847,0,0.8847,0.8803,0.8797,0.6443,0.6512,0.7984,0.1153,0.5569,0


In [44]:
get_incorrect_predictions(predictions_svm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,meanWavelet,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label
371,0.012873,-0.573892,-0.608301,0.586265,-0.043286,-0.772690,0.000642,-0.026207,-0.986468,0.022755,...,-0.687903,-0.276503,0.164883,-0.302537,-0.241933,-0.119511,-0.156183,-0.119637,healthy,damaged
261113,0.036416,-0.036985,-0.018097,0.094470,-0.122904,-0.000295,-0.201324,-0.202111,0.204035,-0.177289,...,0.075207,-0.182138,0.209378,-0.191124,-0.174411,-0.119359,-0.154960,-0.119475,healthy,damaged
207745,0.036146,-0.054912,-0.037686,-0.206209,-0.191703,-0.166550,-0.817718,-0.706493,-0.152971,-0.686273,...,-0.053889,-0.178280,0.195415,-0.197872,-0.181161,-0.119248,-0.154350,-0.119369,healthy,damaged
124580,0.118151,-0.448350,-0.469693,2.502898,0.034874,-0.643918,-0.024141,0.117064,-0.489718,1.077598,...,-0.614637,-0.275118,0.167714,-0.300557,-0.236559,-0.119499,-0.156091,-0.119625,damaged,healthy
82414,0.059542,-0.577416,-0.608461,0.420073,-0.140666,-0.783024,-0.452562,-0.416848,-1.082528,-0.429675,...,-0.692653,-0.276226,0.165140,-0.302350,-0.241967,-0.119511,-0.156183,-0.119637,damaged,healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260610,0.037051,-0.120273,-0.109091,-0.046457,-0.052223,-0.075470,-0.119359,-0.054110,0.157594,0.420565,...,-0.069341,-0.204878,0.201230,-0.220567,-0.194898,-0.119423,-0.155466,-0.119544,healthy,damaged
258687,0.037649,-0.412417,-0.428330,0.099707,0.062800,-0.512874,-0.121624,-0.001592,-0.423838,0.784442,...,-0.523616,-0.262370,0.174837,-0.288400,-0.233108,-0.119503,-0.156112,-0.119628,healthy,damaged
81693,0.065867,-0.578473,-0.609811,-0.018680,-0.113815,-0.777508,-0.068149,-0.135078,-1.034727,-0.366921,...,-0.695291,-0.276252,0.165059,-0.302391,-0.241969,-0.119511,-0.156183,-0.119637,damaged,healthy
205706,0.035491,-0.426794,-0.444088,0.048360,0.038563,-0.511299,0.161131,0.161734,-0.395557,0.467480,...,-0.516059,-0.264963,0.174827,-0.290323,-0.234408,-0.119506,-0.156138,-0.119632,healthy,damaged


# Experiment Setup (DL)

## Configure Data

In [45]:
train, test = train_test_split(features_df_training_normalized, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

Train Shape: (157669, 28) | Val Shape: (39418, 28) | Test Shape: (49272, 28)


In [46]:
target = "Label"

categorical_cols = [
    col
    for col in features_df_training_normalized.select_dtypes(include=["object","category"]).columns
    if col != target
]

continuous_cols = features_df_training_normalized.select_dtypes(include=["number"]).columns.tolist()

In [47]:
print("Target:", target)
print("Categorical inputs:", categorical_cols)  
print("Continuous inputs:", continuous_cols)    

Target: Label
Categorical inputs: []
Continuous inputs: ['mean', 'rms', 'std', 'skewness', 'kurtosis', 'ptp', 'crest', 'impulse', 'clearance', 'shape', 'energy', 'entropy', 'meanFreq', 'medianFreq', 'bandwidth', 'spectral_flatness', 'spectral_entropy', 'spectral_skewness', 'spectral_kurtosis', 'meanWavelet', 'varWavelet', 'entropyWavelet', 'energyWavelet', 'meanSpectrogram', 'varSpectrogram', 'entropySpectrogram', 'energySpectrogram']


In [48]:
data_config = DataConfig(
    target=[target],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

In [49]:
available_gpu=1 if torch.cuda.is_available() else 0
print(f"Available GPU: {'Yes' if available_gpu else 'No'}")

Available GPU: Yes


In [50]:
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=20,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    batch_size=256,
)

optimizer_config = OptimizerConfig()

experiment_config = ExperimentConfig(
        project_name="TEST",
        run_name="test",
        log_target="tensorboard",
    )

In [51]:
n_trials = 10

## Tabnet

In [52]:
def TabNet_Optimization(trial):
    n_d     = trial.suggest_int("n_d", 4, 64)
    n_a     = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma   = trial.suggest_float("gamma", 1.0, 2.0)
    embedding_dropout = trial.suggest_float("embedding_dropout", 0, 1)
    lr      = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
    
    tabnet_config = TabNetModelConfig(
        task="classification",
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        embedding_dropout=embedding_dropout,
        learning_rate=lr,
        n_independent=2,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )
    
    tabnet_model = TabularModel(
        data_config=data_config,
        model_config=tabnet_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    
    tabnet_model.fit(train=train, validation=val)

    preds_df = tabnet_model.predict(val)
    y_pred = preds_df["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [53]:
tabnet_study = optuna.create_study(direction="maximize")
tabnet_study.optimize(TabNet_Optimization, n_trials=n_trials)

2025-09-26 13:18:33,611 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 13:18:33,715 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 13:18:33,746 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 13:18:33,931 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: TabNetModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _embedding_layer │ Identity         │      0 │ train │
│ 1 │ _backbone        │ TabNetBackbone   │  155 K │ train │
│ 2 │ _head            │ Identity         │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 155 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 155 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 107                                                                                         
Modules in eval mode: 0

Output()

2025-09-26 14:59:41,320 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 14:59:41,321 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [54]:
print("Best params:", tabnet_study.best_params)
print("Best F1 score:", tabnet_study.best_value)

Best params: {'n_d': 33, 'n_a': 46, 'n_steps': 3, 'gamma': 1.2864230491528987, 'embedding_dropout': 0.1825276618254278, 'learning_rate': 0.00468046516266798}
Best F1 score: 0.9057275814173009


## GANDALF

In [55]:
def GANDALF_Optimization(trial):
    gflu_stages               = trial.suggest_int("gflu_stages", 1, 10)
    gflu_dropout              = trial.suggest_float("gflu_dropout", 0.0, 0.5)
    gflu_feature_init_sparsity = trial.suggest_float("gflu_feature_init_sparsity", 0.1, 0.9)
    learnable_sparsity        = trial.suggest_categorical("learnable_sparsity", [True, False])
    embedding_dropout         = trial.suggest_float("embedding_dropout", 0.0, 0.5)
    batch_norm_continuous     = trial.suggest_categorical("batch_norm_continuous_input", [True, False])
    learning_rate             = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # 2) build Gandalf config
    gandalf_config = GANDALFConfig(
        task="classification",
        gflu_stages=gflu_stages,
        gflu_dropout=gflu_dropout,
        gflu_feature_init_sparsity=gflu_feature_init_sparsity,
        learnable_sparsity=learnable_sparsity,
        embedding_dropout=embedding_dropout,
        batch_norm_continuous_input=batch_norm_continuous,
        learning_rate=learning_rate,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )

    # 3) instantiate & train
    model = TabularModel(
        data_config=data_config,
        model_config=gandalf_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    model.fit(train=train, validation=val)

    # 4) predict & return macro-F1
    preds = model.predict(val)
    y_pred = preds["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [56]:
gandalf_study = optuna.create_study(direction="maximize")
gandalf_study.optimize(GANDALF_Optimization, n_trials=n_trials)

2025-09-26 14:59:44,664 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 14:59:44,699 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 14:59:44,730 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 14:59:44,897 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │ 26.9 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     54 │ train │
│ 2 │ _head            │ Sequential       │     58 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 27.0 K                                                                                           
Non-trainable params: 6                                                                                            
Total params: 27.0 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 28                                                                                          
Modules in eval mode: 0

Output()

2025-09-26 15:39:35,216 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 15:39:35,218 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [57]:
print("Best params:", gandalf_study.best_params)
print("Best F1 score:", gandalf_study.best_value)

Best params: {'gflu_stages': 8, 'gflu_dropout': 0.20621973444441227, 'gflu_feature_init_sparsity': 0.5069371167197942, 'learnable_sparsity': False, 'embedding_dropout': 0.06647823724375934, 'batch_norm_continuous_input': False, 'learning_rate': 2.6124262014312122e-05}
Best F1 score: 0.9000711214059307
